In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks

### Callbacks as GUI events

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.Button(description= 'Click me')

In [5]:
w

Button(description='Click me', style=ButtonStyle())

hi


In [6]:
def f(o):
    print('hi')

In [7]:
w.on_click(f)

### Creating our own calbacks

In [8]:
from time import sleep

In [9]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [10]:
slow_calculation()

30

In [11]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb:
            cb(i)
    return res

In [12]:
def show_progress(epoch):
    print(f"Awesome! We've finished epoch {epoch} !")

In [13]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0 !
Awesome! We've finished epoch 1 !
Awesome! We've finished epoch 2 !
Awesome! We've finished epoch 3 !
Awesome! We've finished epoch 4 !


30

In [14]:
### Lambdas and partials

In [15]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o} !"))

Awesome! We've finished epoch 0 !
Awesome! We've finished epoch 1 !
Awesome! We've finished epoch 2 !
Awesome! We've finished epoch 3 !
Awesome! We've finished epoch 4 !


30

In [16]:
def show_progress(exclamation, epoch):
    print(f"{exclamation}! We've finished epoch {epoch} !")

In [17]:
slow_calculation(lambda o: show_progress("OK, I guess", o))

OK, I guess! We've finished epoch 0 !
OK, I guess! We've finished epoch 1 !
OK, I guess! We've finished epoch 2 !
OK, I guess! We've finished epoch 3 !
OK, I guess! We've finished epoch 4 !


30

In [18]:
def make_show_progress(exclamation):
    return lambda epoch: print(f"{exclamation}! We've finished epoch {epoch} !")

In [19]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We've finished epoch 0 !
Nice!! We've finished epoch 1 !
Nice!! We've finished epoch 2 !
Nice!! We've finished epoch 3 !
Nice!! We've finished epoch 4 !


30

In [20]:
from functools import partial

In [21]:
slow_calculation(partial(show_progress,"ok. I guess"))

ok. I guess! We've finished epoch 0 !
ok. I guess! We've finished epoch 1 !
ok. I guess! We've finished epoch 2 !
ok. I guess! We've finished epoch 3 !
ok. I guess! We've finished epoch 4 !


30

In [22]:
f2 =  partial(show_progress,"ok. I guess")

### Callbacks as callable classes

In [23]:
class ProgressShowingCallback():
    def __init__(self, exclamation='Awesome'):
        self.exclamation =  exclamation
    def __call__(self, epoch):
        print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [24]:
cb = ProgressShowingCallback("Just super")

In [25]:
slow_calculation(cb)

Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

### Multiple callback funcs ; *args and **kwargs

In [27]:
def f(*args, **kwargs):
    print(f"args: {args}; kwargs: {kwargs}")

In [28]:
f(3, 'a', thing1="hello")

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [29]:
def g(a,b, c=0):
    print(a,b,c)

In [30]:
args = [1,2]
kwargs = {'c':3}
g(*args, **kwargs)

1 2 3


In [36]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val = res)
    return res

In [37]:
class PrintStepCallback():
    def __init__(self):
        pass
    def before_calc(self, *args, **kwargs): 
        print(f"about to start")
    def after_calc(self, *args, **kwargs):
        print(f"Done step")

In [38]:
slow_calculation(PrintStepCallback())

about to start
Done step
about to start
Done step
about to start
Done step
about to start
Done step
about to start
Done step


30

In [42]:
class PrintStatusCallback():
    def __init__(self):
        pass
    def before_calc(self, epoch, **kwargs): 
        print(f"about to start: {epoch}")
    def after_calc(self, epoch, val, **kwargs):
        print(f"After {epoch}: {val}")

In [43]:
slow_calculation(PrintStatusCallback())

about to start: 0
After 0: 0
about to start: 1
After 1: 1
about to start: 2
After 2: 5
about to start: 3
After 3: 14
about to start: 4
After 4: 30


30

### __dunder__

In [48]:
class SloppyAdder():
    def __init__(self, o):
        self.o = o
    def __add__(self,b):
        return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self):
        return str(self.o)

In [49]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

Special methods you should probably know about (see data model link above) are:

- `__getitem__`
- `__getattr__`
- `__setattr__`
- `__del__`
- `__init__`
- `__new__`
- `__enter__`
- `__exit__`
- `__len__`
- `__repr__`
- `__str__`

## __get_attr__ and getattr

In [53]:
class A: 
    a,b= 1,2

In [54]:
a = A()

In [55]:
a.b

2

In [56]:
getattr(a, 'b')

2

In [63]:
getattr(a, 'b' if random.random() > 0.5 else 'a')

1

In [64]:
class B:
    a,b = 1,2
    def __getattr__(self,k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello from {k}'

In [65]:
b = B()

In [66]:
b.a

1

In [67]:
b.foo

'Hello from foo'

In [68]:
b.foo

'Hello from foo'

In [69]:
b.a

1